In [63]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import scipy.sparse as sparse

def biased_als(data, iterations, lmbda, features):
    user_size, item_size = data.shape
    
    
    X = sparse.csr_matrix(np.random.normal(size = (user_size, features)))
    Y = sparse.csr_matrix(np.random.normal(size = (item_size, features)))
    
    beta = np.zeros((1, user_size))
    gamma = np.zeros((item_size, 1))
    
    for _ in tqdm(range(iterations)):
        for i in range(item_size):
            r_i = data[:, i].T.toarray()
            r_i_beta = r_i - beta
            
            X = (sparse.hstack([np.ones((user_size, 1)), X])).T
            Y = sparse.hstack([gamma, Y])
            
            print('X.shape', X.shape)
            print('r_i_beta.shape', r_i_beta.shape)
            
            Y[i] = (X.T @ X + lmbda * sparse.eye(features+1)) ** -1 
            print('Y[i].shape', Y[i].shape)
            Y[i] @= X.T @ r_i_beta
            
        
        for u in range(user_size):
            
            r_u = data[u, :].toarray()
            
            r_u_gamma = r_u - gamma
            
            Y = sparse.hstack([np.ones((1, item_size)), Y])
            
            X = (sparse.hstack([beta, X.T])).T
            
            X[u] = (Y.T @ Y + lmbda*I)**-1 @ Y.T @ r_u_gamma
            
    return X, Y

In [2]:
df_ratings = pd.read_csv('ml-1m/ratings.dat', sep="::", header=None)
df_ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

C:\Users\Andrey\AppData\Roaming\Python\Python38\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
df_ratings.drop('timestamp', axis='columns', inplace=True)
df_ratings.dropna(inplace=True)

In [4]:
df_ratings['movie_num'] = df_ratings['movie_id'].astype("category").cat.codes
item_lookup = df_ratings[['movie_num', 'movie_id']].drop_duplicates()
item_lookup['movie_num'] = item_lookup.movie_num.astype(str)

In [5]:
users = list(np.sort(df_ratings.user_id.unique()))
movies = list(np.sort(df_ratings.movie_num.unique()))
ratings = list(df_ratings.rating)

In [6]:
rows = df_ratings.user_id.astype(int)
cols = df_ratings.movie_num.astype(int)

In [7]:
data_sparse = sparse.csr_matrix(
    (ratings, (rows-1, cols)),
    shape=(len(users), len(movies)))

In [64]:
user_vecs, item_vecs = biased_als(data_sparse, iterations=20, features=20, lmbda=10)

  0%|          | 0/20 [00:00<?, ?it/s]

X.shape (21, 6040)
r_i_beta.shape (1, 6040)


ValueError: exponent must be >= 0